#### 1. Downloading MS-COCO Dataset

In [1]:
!mkdir coco
!cd coco
!mkdir images
!cd images

!wget -c http://images.cocodataset.org/zips/train2017.zip


!unzip train2017.zip

!rm train2017.zip

!cd ../
!wget -c http://images.cocodataset.org/annotations/annotations_trainval2017.zip
!wget -c http://images.cocodataset.org/annotations/stuff_annotations_trainval2017.zip

!unzip annotations_trainval2017.zip
!unzip stuff_annotations_trainval2017.zip

!rm annotations_trainval2017.zip
!rm stuff_annotations_trainval2017.zip


Streaming output truncated to the last 5000 lines.
 extracting: train2017/000000395397.jpg  
 extracting: train2017/000000255134.jpg  
 extracting: train2017/000000549188.jpg  
 extracting: train2017/000000383267.jpg  
 extracting: train2017/000000439710.jpg  
 extracting: train2017/000000041920.jpg  
 extracting: train2017/000000135984.jpg  
 extracting: train2017/000000530786.jpg  
 extracting: train2017/000000439236.jpg  
 extracting: train2017/000000485526.jpg  
 extracting: train2017/000000384625.jpg  
 extracting: train2017/000000369986.jpg  
 extracting: train2017/000000348913.jpg  
 extracting: train2017/000000328306.jpg  
 extracting: train2017/000000340314.jpg  
 extracting: train2017/000000515485.jpg  
 extracting: train2017/000000402909.jpg  
 extracting: train2017/000000482195.jpg  
 extracting: train2017/000000557251.jpg  
 extracting: train2017/000000136487.jpg  
 extracting: train2017/000000404064.jpg  
 extracting: train2017/000000511774.jpg  
 extracting: train2017/00

#### 2. Training Stage 1 of GAN
> Steps
1. Generate a batch of text descriptions and corresponding low res images from pre-processed dataset.
2. Pass the text descriptions encoded via Encoder model through generator model to generate low-res images.
3. Train discriminator model on batch of real low res and generated low res images.
4. Train generator model on batch of generated low res images and text description by optimizing loss function.
5. Repeat 1 -4 for a fixed number of epochs to improve the generator and discriminator models.



In [11]:
# mandatory imports
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from nltk.tokenize import RegexpTokenizer
from PIL import Image
import numpy as np
import json
!pip install torch-summary --quiet
import torchsummary
import torchvision.utils as vutils


In [3]:
# defining the stage 1 generator and discriminator models

class Stage1Generator(nn.Module):
    def __init__(self):
        super(Stage1Generator, self).__init__()
        self.fc = nn.Linear(100, 256 * 8 * 8)
        self.bn1 = nn.BatchNorm2d(256)
        self.up1 = nn.Upsample(scale_factor=2)
        self.conv1 = nn.Conv2d(256, 128, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(128)
        self.up2 = nn.Upsample(scale_factor=2)
        self.conv2 = nn.Conv2d(128, 64, 3, padding=1)
        self.bn3 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 3, 3, padding=1)
        self.up3 = nn.Upsample(scale_factor=2)

    def forward(self, z):
        x = self.fc(z)
        x = x.view(-1, 256, 8, 8)
        x = self.bn1(x)
        x = self.up1(x)
        x = self.conv1(x)
        x = self.bn2(x)
        x = self.up2(x)
        x = self.conv2(x)
        x = self.bn3(x)
        x = torch.tanh(self.up3(self.conv3(x)))
        return x

In [5]:
# the discriminator takes in the rgb image and predicts fake or real
class Stage1Discriminator(nn.Module):
    def __init__(self):
        super(Stage1Discriminator, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, 4, stride=2, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        self.conv2 = nn.Conv2d(64, 128, 4, stride=2, padding=1)
        self.bn2 = nn.BatchNorm2d(128)
        self.conv3 = nn.Conv2d(128, 256, 4, stride=2, padding=1)
        self.bn3 = nn.BatchNorm2d(256)
        self.conv4 = nn.Conv2d(256, 512, 4, padding=1)
        self.bn4 = nn.BatchNorm2d(512)
        self.conv5 = nn.Conv2d(512, 1, 4, padding=1)

    def forward(self, x):
        x = nn.functional.leaky_relu(self.bn1(self.conv1(x)), 0.2)
        x = nn.functional.leaky_relu(self.bn2(self.conv2(x)), 0.2)
        x = nn.functional.leaky_relu(self.bn3(self.conv3(x)), 0.2)
        x = nn.functional.leaky_relu(self.bn4(self.conv4(x)), 0.2)
        x = torch.sigmoid(self.conv5(x))
        return x.view(-1, 1)

In [6]:
# defining the encoder module 
# encodes the image to 
class Encoder(nn.Module):
    def __init__(self, vocab_size, embedding_size, hidden_size, num_layers):
        super(Encoder, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_size)
        self.lstm = nn.LSTM(embedding_size, hidden_size, num_layers=num_layers, bidirectional=True)
        self.fc = nn.Linear(hidden_size*2, 100) # output size of 100
    
    def forward(self, captions):
        # captions: (batch_size, seq_len)
        embedded = self.embedding(captions) # embedded: (batch_size, seq_len, embedding_size)
        lstm_out, _ = self.lstm(embedded) # lstm_out: (batch_size, seq_len, hidden_size*2)
        lstm_out = lstm_out.mean(dim=1) # lstm_out: (batch_size, hidden_size*2)
        encoded = self.fc(lstm_out) # encoded: (batch_size, 100)
        encoded = F.normalize(encoded, p=2, dim=1) # normalize the encoded output
        return encoded
        


In [16]:
# Defining the dataset and datalaoder

class CocoDataset(Dataset):
    def __init__(self, captions_path, image_folder, transform=None, vocab_size=5000, embed_dim=300):
        self.image_folder = image_folder
        self.transform = transform
        
        with open(captions_path) as f:
            self.captions_data = json.load(f)['annotations']
        
        self.tokenizer = RegexpTokenizer(r'\w+')
        
        self.word2idx = {}
        self.idx2word = {}
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim
        
        self.build_vocab()
    
    def build_vocab(self):
        # Count word frequency
        word_freq = {}
        for caption in self.captions_data:
            tokens = self.tokenizer.tokenize(caption['caption'].lower())
            for token in tokens:
                if token not in word_freq:
                    word_freq[token] = 1
                else:
                    word_freq[token] += 1
        
        # Sort words by frequency
        sorted_words = sorted(word_freq.items(), key=lambda x: x[1], reverse=True)
        sorted_words = [word for word, freq in sorted_words][:self.vocab_size-2]
        self.word2idx = {'<PAD>': 0, '<UNK>': 1}
        self.idx2word = {0: '<PAD>', 1: '<UNK>'}
        for i, word in enumerate(sorted_words):
            self.word2idx[word] = i+2
            self.idx2word[i+2] = word
        
    def caption_to_vector(self, caption):
        tokens = self.tokenizer.tokenize(caption.lower())
        caption_vector = torch.zeros(len(tokens)+2, dtype=torch.long)
        caption_vector[0] = self.word2idx['<PAD>']
        caption_vector[-1] = self.word2idx['<PAD>']
        for i, token in enumerate(tokens):
            if token in self.word2idx:
                caption_vector[i+1] = self.word2idx[token]
            else:
                caption_vector[i+1] = self.word2idx['<UNK>']
        return caption_vector
    
    def __getitem__(self, index):
        caption_data = self.captions_data[index]
        image_id = caption_data['image_id']
        caption = caption_data['caption']
        caption_vector = self.caption_to_vector(caption)
        
        image_path = f'{self.image_folder}/{str(image_id).zfill(12)}.jpg'
        image = Image.open(image_path).convert('RGB')
        
        if self.transform is not None:
            image = self.transform(image)
        
        return image, caption_vector
    
    def __len__(self):
        return len(self.captions_data)
    
def collate_fn(data):
    # Sort data by caption length (in descending order) to use pack_padded_sequence
    data.sort(key=lambda x: len(x[1]), reverse=True)
    images, captions = zip(*data)
    images = torch.stack(images, 0)

    # Pad captions to max length
    lengths = [len(cap) for cap in captions]
    targets = torch.zeros(len(captions), max(lengths)).long()
    for i, cap in enumerate(captions):
        end = lengths[i]
        targets[i, :end] = torch.LongTensor(cap[:end])
    
    return images, targets, lengths

# Set parameters
img_dir = '/content/train2017'
caption_path = '/content/annotations/captions_train2017.json'

# Define transforms for images
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

batch_size=1024
# Create COCODataset object
coco_dataset = CocoDataset(caption_path, img_dir, transform=transform)
dataloader = DataLoader(coco_dataset, batch_size=batch_size, shuffle=True, num_workers=2, collate_fn=collate_fn)

In [17]:
for batch in dataloader:
  print(batch)
  break

(tensor([[[[ 0.2784,  0.2235,  0.0667,  ..., -0.4196, -0.4275, -0.4431],
          [ 0.2784,  0.2784,  0.2392,  ..., -0.4039, -0.4118, -0.4275],
          [ 0.2392,  0.2314,  0.2706,  ..., -0.3882, -0.3961, -0.4118],
          ...,
          [-0.6627, -0.6471, -0.6392,  ..., -0.3098, -0.3725, -0.5843],
          [-0.6863, -0.6784, -0.6627,  ..., -0.3333, -0.3882, -0.5765],
          [-0.6941, -0.6784, -0.6784,  ..., -0.3020, -0.3725, -0.5608]],

         [[-0.1137, -0.1686, -0.3412,  ..., -0.4745, -0.4902, -0.5137],
          [-0.1059, -0.0745, -0.1373,  ..., -0.4510, -0.4824, -0.4902],
          [-0.1686, -0.1529, -0.1294,  ..., -0.4431, -0.4588, -0.4745],
          ...,
          [-0.3961, -0.3647, -0.3333,  ..., -0.4588, -0.4902, -0.6471],
          [-0.4196, -0.3882, -0.3647,  ..., -0.4745, -0.4902, -0.6471],
          [-0.4510, -0.4118, -0.3961,  ..., -0.4431, -0.4902, -0.6392]],

         [[-0.2157, -0.2941, -0.4667,  ..., -0.4510, -0.4588, -0.4824],
          [-0.2078, -0.1922, 

In [37]:
vocab_size = 5000  # size of the vocabulary, which can be set to the number of unique words in the dataset
embedding_size = 300  # size of the word embedding, which can be set to a reasonable value (e.g. 300)
hidden_size = 512  # size of the LSTM hidden state, which can be set to a reasonable value (e.g. 512)
num_layers = 2  # number of LSTM layers, which can be set to a reasonable value (e.g. 2)

In [14]:
# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Initialize networks and optimizers
G1 = Stage1Generator().to(device)
D1 = Stage1Discriminator().to(device)
E = Encoder(vocab_size, embedding_size, hidden_size, num_layers).to(device)
G1_optimizer = optim.Adam(G1.parameters(), lr=0.0002, betas=(0.5, 0.999))
D1_optimizer = optim.Adam(D1.parameters(), lr=0.0002, betas=(0.5, 0.999))
E_optimizer = optim.Adam(E.parameters(), lr=0.0002, betas=(0.5, 0.999))

In [20]:
torchsummary.summary(G1,input_data=(32,100))

Layer (type:depth-idx)                   Output Shape              Param #
├─Linear: 1-1                            [-1, 32, 16384]           1,654,784
├─BatchNorm2d: 1-2                       [-1, 256, 8, 8]           512
├─Upsample: 1-3                          [-1, 256, 16, 16]         --
├─Conv2d: 1-4                            [-1, 128, 16, 16]         295,040
├─BatchNorm2d: 1-5                       [-1, 128, 16, 16]         256
├─Upsample: 1-6                          [-1, 128, 32, 32]         --
├─Conv2d: 1-7                            [-1, 64, 32, 32]          73,792
├─BatchNorm2d: 1-8                       [-1, 64, 32, 32]          128
├─Conv2d: 1-9                            [-1, 3, 32, 32]           1,731
├─Upsample: 1-10                         [-1, 3, 64, 64]           --
Total params: 2,026,243
Trainable params: 2,026,243
Non-trainable params: 0
Total mult-adds (M): 154.40
Input size (MB): 0.01
Forward/backward pass size (MB): 5.65
Params size (MB): 7.73
Estimated Total 

Layer (type:depth-idx)                   Output Shape              Param #
├─Linear: 1-1                            [-1, 32, 16384]           1,654,784
├─BatchNorm2d: 1-2                       [-1, 256, 8, 8]           512
├─Upsample: 1-3                          [-1, 256, 16, 16]         --
├─Conv2d: 1-4                            [-1, 128, 16, 16]         295,040
├─BatchNorm2d: 1-5                       [-1, 128, 16, 16]         256
├─Upsample: 1-6                          [-1, 128, 32, 32]         --
├─Conv2d: 1-7                            [-1, 64, 32, 32]          73,792
├─BatchNorm2d: 1-8                       [-1, 64, 32, 32]          128
├─Conv2d: 1-9                            [-1, 3, 32, 32]           1,731
├─Upsample: 1-10                         [-1, 3, 64, 64]           --
Total params: 2,026,243
Trainable params: 2,026,243
Non-trainable params: 0
Total mult-adds (M): 154.40
Input size (MB): 0.01
Forward/backward pass size (MB): 5.65
Params size (MB): 7.73
Estimated Total 

In [21]:
D1

Stage1Discriminator(
  (conv1): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): Conv2d(256, 512, kernel_size=(4, 4), stride=(1, 1), padding=(1, 1))
  (bn4): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv5): Conv2d(512, 1, kernel_size=(4, 4), stride=(1, 1), padding=(1, 1))
)

In [22]:
E

Encoder(
  (embedding): Embedding(5000, 300)
  (lstm): LSTM(300, 512, num_layers=2, bidirectional=True)
  (fc): Linear(in_features=1024, out_features=100, bias=True)
)

In [23]:
# Define loss functions
criterion = nn.BCELoss()
mse_loss = nn.MSELoss()

In [24]:
# Training loop
num_epochs=10
for epoch in range(num_epochs):
    for i, data in enumerate(dataloader):
        real_images = data[0]
        captions = data[1]
        captions = captions.to(device)
        real_images = real_images.to(device)

        # Train Discriminator
        D1_optimizer.zero_grad()
        fake_images = G1(torch.randn(real_images.size(0), 100).to(device)) # encoder generates batch_size,100 image which is being taken by generator
        d_real = D1(real_images)
        d_fake = D1(fake_images.detach())
        D1_loss = criterion(d_real, torch.ones_like(d_real)) + criterion(d_fake, torch.zeros_like(d_fake))
        D1_loss.backward()
        D1_optimizer.step()

        # Train Encoder and Stage 1 Generator
        G1_optimizer.zero_grad()
        E_optimizer.zero_grad()
        encoded_real = E(captions)
        #print(encoded_real.shape)
        #print(encoded_real.shape)
        fake_images_encoded = G1(encoded_real)
        d_fake_encoded = D1(fake_images_encoded)
        G1_loss = criterion(d_fake_encoded, torch.ones_like(d_fake_encoded))
        #print(fake_images_encoded.shape, real_images.shape)
        mse_loss_encoded = mse_loss(fake_images_encoded, real_images)
        E_loss = mse_loss_encoded
        G1_E_loss = G1_loss + 0.01 * mse_loss_encoded
        G1_E_loss.backward()
        G1_optimizer.step()
        E_optimizer.step()

        # Print losses
        if (i+1) % 5 == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(dataloader)}], D1_loss: {D1_loss.item():.4f}, G1_loss: {G1_loss.item():.4f}, E_loss: {E_loss.item():.4f}")
    
    #saving models
    torch.save(G1.state_dict(), f"stage1_generator_epoch_{epoch+1}.pth")
    torch.save(D1.state_dict(), f"stage1_discriminator_epoch_{epoch+1}.pth")
    torch.save(E.state_dict(), f"encoder_epoch_{epoch+1}.pth")


Epoch [1/10], Step [5/578], D1_loss: 1.2320, G1_loss: 0.4827, E_loss: 0.5979
Epoch [1/10], Step [10/578], D1_loss: 0.8807, G1_loss: 0.1282, E_loss: 0.6776
Epoch [1/10], Step [15/578], D1_loss: 0.5196, G1_loss: 0.0178, E_loss: 0.7061
Epoch [1/10], Step [20/578], D1_loss: 0.1836, G1_loss: 0.0065, E_loss: 0.6984
Epoch [1/10], Step [25/578], D1_loss: 0.0845, G1_loss: 0.0033, E_loss: 0.6935
Epoch [1/10], Step [30/578], D1_loss: 0.0521, G1_loss: 0.0022, E_loss: 0.6818
Epoch [1/10], Step [35/578], D1_loss: 0.0355, G1_loss: 0.0016, E_loss: 0.6668
Epoch [1/10], Step [40/578], D1_loss: 0.0250, G1_loss: 0.0012, E_loss: 0.6511
Epoch [1/10], Step [45/578], D1_loss: 0.0202, G1_loss: 0.0010, E_loss: 0.6294
Epoch [1/10], Step [50/578], D1_loss: 0.0159, G1_loss: 0.0009, E_loss: 0.6222
Epoch [1/10], Step [55/578], D1_loss: 0.0125, G1_loss: 0.0008, E_loss: 0.6000
Epoch [1/10], Step [60/578], D1_loss: 0.0110, G1_loss: 0.0007, E_loss: 0.6051
Epoch [1/10], Step [65/578], D1_loss: 0.0095, G1_loss: 0.0006, E_

KeyboardInterrupt: ignored

##### Loading pre-trained models



In [24]:
# Load pre-trained models
G1.load_state_dict(torch.load("stage1_generator_epoch_1.pth",map_location=torch.device('cpu')))
D1.load_state_dict(torch.load("stage1_discriminator_epoch_1.pth",map_location=torch.device('cpu')))
E.load_state_dict(torch.load("encoder_epoch_1.pth",map_location=torch.device('cpu')))

<All keys matched successfully>

#### Prediction 

In [36]:
# Generate word to index.json for coco
import json
from collections import Counter
from pycocotools.coco import COCO

def generate_word_to_index_coco(coco_annotation_file, word_frequency_threshold=5):
    # Initialize COCO API
    coco = COCO(coco_annotation_file)
    # Load all captions from the dataset
    captions = coco.imgToAnns
    # Count the frequency of each word in the captions
    word_frequency = Counter()
    for image_id in captions:
        for caption in captions[image_id]:
            tokens = caption['caption'].lower().split()
            word_frequency.update(tokens)
    # Filter out infrequent words
    vocab = set()
    for word, frequency in word_frequency.items():
        if frequency >= word_frequency_threshold:
            vocab.add(word)
    # Create word-to-index mapping
    word_to_index = {'<PAD>': 0, '<UNK>': 1}
    index = 2
    for word in sorted(vocab):
        word_to_index[word] = index
        index += 1
    # Save the word-to-index mapping to a file
    with open('/content/word_to_index_coco.json', 'w') as f:
        json.dump(word_to_index, f)

generate_word_to_index_coco('/content/annotations/captions_train2017.json')

loading annotations into memory...
Done (t=1.12s)
creating index...
index created!


In [44]:
def get_text_embedding(caption, embedding_dim=300, hidden_dim=512, device='cpu'):
    """Converts a caption to a text embedding tensor using TextEncoder."""
    # Load the word-to-index mapping
    with open('/content/word_to_index_coco.json', 'r') as f:
        word_to_index = json.load(f)
    # Convert caption to lowercase and split into words
    caption = caption.lower().split()
    # Convert words to indices
    caption_indices = [word_to_index.get(word, 0) for word in caption]
    # Pad or truncate caption indices to a fixed length
    max_caption_length = 20
    caption_indices = caption_indices[:max_caption_length] + [0] * (max_caption_length - len(caption_indices))
    # Convert caption indices to a tensor
    caption_tensor = torch.tensor(caption_indices).unsqueeze(0)
    # Initialize TextEncoder
    text_encoder = Encoder(5000, embedding_dim, hidden_dim, 2) # vocab_size=len(word_to_index)
    text_encoder.load_state_dict(torch.load('encoder_epoch_1.pth', map_location=device))
    text_encoder = text_encoder.to(device)
    text_encoder.eval()
    # Encode the caption tensor to a text embedding tensor
    with torch.no_grad():
        text_embedding = text_encoder(caption_tensor.to(device))
    return text_embedding



In [63]:
caption = "dairy farmer"
z = get_text_embedding(caption)
print(z.shape)
fake_images = G1(z)

vutils.save_image(fake_images.data, '/content/output_stage1.png', normalize=True)



torch.Size([1, 100])


#### 2. Training Stage 2 of GAN
> Steps
1. Load the Stage 1 generator, discriminator and encoder modles.
2. Freeze the parameters of the Stage 1 models, so they are not updated during training.
3. Define the stage 2 geneatrator and discriminator models.
4. Define the optimizers for the Stage 2 models.
5. Load the dataset and create dataloaders for training.
6. In each iteration
- Sample a batch of real images and captions from dataset.
- Generate fake images from stage 2 generator using captions encoded via Encoder model as input.
- Encode the real and fake iamges using stage 1 encoder.
- compute the stage 2 discriminator loss on the real and fake image embeddings.
- update stage 2 discriminator params during back prop and discriminator optimizer step.
- generate fake images from stage 2 generators using captions as input
- encode the fake images using stage 1 encoder
- compute stage 2 generator loss on fake image embeddings.
- update the stage 22 generator params during backprop

7. save the stage 2 generator and disc models
8. repeat till model loss converges




In [ ]:
# Freeze the parameters of the Stage 1 models
for param in stage1_generator.parameters():
    param.requires_grad = False
for param in stage1_discriminator.parameters():
    param.requires_grad = False
for param in stage1_encoder.parameters():
    param.requires_grad = False


In [ ]:
# defining stage 2 generator and discriminator models along with encoder
#The Stage 2 Generator takes as input the concatenated vector of noise and the Stage 1 generated image, and it outputs a 256x256 RGB image. 
# i.e Stage 2 generator: Takes in the concatenation of the low-resolution image generated by stage 1 generator and the text embedding vector generated by the stage 1 encoder. The output is a high-resolution image.

class Stage2Generator(nn.Module):
    def __init__(self):
        super(Stage2Generator, self).__init__()
        self.fc1 = nn.Linear(110, 1024)
        self.bn1 = nn.BatchNorm1d(1024)
        self.fc2 = nn.Linear(1024, 128 * 16 * 16)
        self.bn2 = nn.BatchNorm1d(128 * 16 * 16)
        self.up1 = nn.Upsample(scale_factor=2)
        self.conv1 = nn.Conv2d(128, 64, 3, padding=1)
        self.bn3 = nn.BatchNorm2d(64)
        self.up2 = nn.Upsample(scale_factor=2)
        self.conv2 = nn.Conv2d(64, 32, 3, padding=1)
        self.bn4 = nn.BatchNorm2d(32)
        self.conv3 = nn.Conv2d(32, 3, 3, padding=1)
        self.tanh = nn.Tanh()

    def forward(self, z, c):
        x = torch.cat([z, c], dim=1)
        x = nn.functional.relu(self.bn1(self.fc1(x)))
        x = nn.functional.relu(self.bn2(self.fc2(x)))
        x = x.view(-1, 128, 16, 16)
        x = self.up1(x)
        x = nn.functional.relu(self.bn3(self.conv1(x)))
        x = self.up2(x)
        x = nn.functional.relu(self.bn4(self.conv2(x)))
        x = self.tanh(self.conv3(x))
        return x

In [ ]:
#Stage 2 discriminator: Takes in a high-resolution image and predicts fake or real.
class Stage2Discriminator(nn.Module):
    def __init__(self):
        super(Stage2Discriminator, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 4, stride=2, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, 4, stride=2, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, 4, stride=2, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.conv4 = nn.Conv2d(128, 256, 4, stride=2, padding=1)
        self.bn4 = nn.BatchNorm2d(256)
        self.conv5 = nn.Conv2d(256, 1, 4, padding=0)

    def forward(self, x, c):
        c = c.view(-1, 10, 1, 1)
        c = c.repeat(1, 1, 4, 4)
        x = torch.cat([x, c], dim=1)
        x = nn.functional.leaky_relu(self.bn1(self.conv1(x)), 0.2)
        x = nn.functional.leaky_relu(self.bn2(self.conv2(x)), 0.2)
        x = nn.functional.leaky_relu(self.bn3(self.conv3(x)), 0.2)
        x = nn.functional.leaky_relu(self.bn4(self.conv4(x)), 0.2)
        x = self.conv5(x)
        return x.view(-1, 1)

In [ ]:
#Stage 2 encoder: Takes in a high-resolution image and generates the text embedding vector.
#In Stage2Encoder, we first pass the captions through the Encoder module defined earlier to get an encoded representation of the input text. We then pass this encoded representation through two fully connected layers with output sizes of 128, to obtain the mean and log variance of the Gaussian distribution that we use to sample the noise vector z. We use the reparameterization trick to sample z from the Gaussian distribution, and return it as the output of the module.
#This encoder has the same architecture as the one used in the first stage, but with some differences:
#1. The output of the LSTM layer is fed into a fully connected layer with 256 units and ReLU activation.
#2. The output of this layer is then fed into another fully connected layer with 100 units (the same size as the noise vector used in the generator of the first stage).
#3. The output of this layer is normalized using L2 normalization, just like in the first stage.

class Stage2Encoder(nn.Module):
    def __init__(self, vocab_size, embedding_size, hidden_size, num_layers):
        super(Stage2Encoder, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_size)
        self.lstm = nn.LSTM(embedding_size, hidden_size, num_layers=num_layers, bidirectional=True)
        self.fc1 = nn.Linear(hidden_size*2, 256)
        self.fc2 = nn.Linear(256, 100) # output size of 100
        
    def forward(self, captions):
        # captions: (batch_size, seq_len)
        embedded = self.embedding(captions) # embedded: (batch_size, seq_len, embedding_size)
        lstm_out, _ = self.lstm(embedded) # lstm_out: (batch_size, seq_len, hidden_size*2)
        lstm_out = lstm_out.mean(dim=1) # lstm_out: (batch_size, hidden_size*2)
        x = nn.functional.relu(self.fc1(lstm_out)) # x: (batch_size, 256)
        encoded = F.normalize(self.fc2(x), p=2, dim=1) # encoded: (batch_size, 100)
        return encoded

In [ ]:
stage2_generator = Stage2Generator().to(device)
stage2_discriminator = Stage2Discriminator().to(device)
stage2_encoder = Stage2Encoder(coco_dataset.vocab_size, 256, 256, 2).to(device)

In [ ]:
# define optimizers for each network
gen_optimizer = optim.Adam(stage2_generator.parameters(), lr=0.0002, betas=(0.5, 0.999))
disc_optimizer = optim.Adam(stage2_discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))
enc_optimizer = optim.Adam(stage2_encoder.parameters(), lr=0.0002, betas=(0.5, 0.999))

# define loss function
adversarial_loss = nn.BCELoss()
auxiliary_loss = nn.CrossEntropyLoss()

